In [2]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import netCDF4 as nc
import geopandas as gpd
import rioxarray as rxr
from myfunc import timer
from myfunc import DirMan
from myfunc import load_and_flatten_data
import config

resolution     = config.resolution
region         = config.region
data_path      = config.data_path
post_data_path = config.post_data_path
shp_path       = config.shp_path

dir_man = DirMan(data_path)
dir_man.enter()

os.makedirs(f'{data_path}/csv', exist_ok=True)

Entered /tera11/zhwei/students/Xionghui/data/run/0p1/


In [2]:
df = pd.read_csv(f'{post_data_path}/field/field_all.csv', encoding='latin-1')
df

# dir_man = DirMan(data_path)
# dir_man.enter()

s1 = nc.Dataset('Sbedrock_tmp2.nc4', 'r')
s2 = nc.Dataset('Ssoil.nc4', 'r')
s3 = nc.Dataset('DTB.nc4', 'r')
s4 = nc.Dataset('mask1.nc4', 'r')
s5 = nc.Dataset('mask2.nc4', 'r')
s6 = nc.Dataset('mask3.nc4', 'r')
s7 = nc.Dataset('mask123.nc4', 'r')

lat = df['Latitude']
lon = df['Longitude']
ssa = df['Same Site As']
# print(lat,lon)

lat1 = s1.variables['lat'][:]
lon1 = s1.variables['lon'][:]
# print(lat1,lon1)

df2 = pd.DataFrame()
j = 0
for i in range(len(lat)):
    if not isinstance(ssa[i], str):
        # print('-----------------------------------------------')
        
        lat1_index = np.argmin(np.abs(lat1 - lat[i]))
        lon1_index = np.argmin(np.abs(lon1 - lon[i]))
        lat1_target = lat1[lat1_index]
        lon1_target = lon1[lon1_index]
        # print(lat[i])
        # print(lon[i])
        # print('find ')
        # print(lat1_target)
        # print(lon1_target)
        # print('-----------------------------------------------')
        
        df2.loc[j, 'Measure'] = df.loc[i, 'Measurement or Estimate of RM Contribution to ET?']
        df2.loc[j, 'lat'] = lat[i]
        df2.loc[j, 'lon'] = lon[i]
        df2.loc[j, 'Sbedrock_field_min'] = df.loc[i, 'Minimum']
        df2.loc[j, 'Sbedrock_field_max'] = df.loc[i, 'Maximum']
        df2.loc[j, 'Sbedrock'] = s1['Sbedrock'][lat1_index,lon1_index]
        df2.loc[j, 'Ssoil'] = s2['Band1'][lat1_index,lon1_index]
        df2.loc[j, 'Soil_depth'] = df.loc[i, 'SoilDepth_Numberline_cm']
        df2.loc[j, 'DTB'] = s3['Band1'][lat1_index,lon1_index]
        df2.loc[j, 'mask1'] = s4['Band1'][lat1_index,lon1_index]
        df2.loc[j, 'mask2'] = s5['LC'][0,lat1_index,lon1_index]
        df2.loc[j, 'mask3'] = s6['et'][0,lat1_index,lon1_index]
        df2.loc[j, 'mask'] = s7['Band1'][lat1_index,lon1_index]
        
        j += 1

df3 = df2.groupby(['lat', 'lon']).first().reset_index()
# df3.fillna(0, inplace=True)
df3['num'] = range(len(df3['lat']))
with open('csv/site.csv','w') as f:
    df3.to_csv(f)
    
# print(df3)
df3

/home/xuxh22/anaconda3/envs/jupyter/lib/python3.10/site-packages/pandas/core/internals/blocks.py:1429: UserWarning: Warning: converting a masked element to nan.
  values[indexer] = casted
/home/xuxh22/anaconda3/envs/jupyter/lib/python3.10/site-packages/pandas/core/internals/blocks.py:1429: UserWarning: Warning: converting a masked element to nan.
  values[indexer] = casted
/home/xuxh22/anaconda3/envs/jupyter/lib/python3.10/site-packages/pandas/core/internals/blocks.py:1429: UserWarning: Warning: converting a masked element to nan.
  values[indexer] = casted
/home/xuxh22/anaconda3/envs/jupyter/lib/python3.10/site-packages/pandas/core/internals/blocks.py:1429: UserWarning: Warning: converting a masked element to nan.
  values[indexer] = casted
/home/xuxh22/anaconda3/envs/jupyter/lib/python3.10/site-packages/pandas/core/internals/blocks.py:1429: UserWarning: Warning: converting a masked element to nan.
  values[indexer] = casted
/home/xuxh22/anaconda3/envs/jupyter/lib/python3.10/site-pack

,lat,lon,Measure,Sbedrock_field_min,Sbedrock_field_max,Sbedrock,Ssoil,Soil_depth,DTB,mask1,mask2,mask3,mask,num
0,-34.300000,19.150000,N,NaN,NaN,147.998795,207.693985,60.0,599.061157,NaN,1.0,NaN,NaN,0
1,-32.338300,116.090400,N,NaN,NaN,300.742432,75.005013,NaN,100.000000,1.0,1.0,NaN,NaN,1
2,-30.412800,18.249700,N,NaN,NaN,67.932983,69.475952,0.3,100.000000,1.0,1.0,NaN,NaN,2
3,11.730000,76.450000,Y,NaN,NaN,199.571167,98.559509,NaN,100.000000,1.0,1.0,1.0,1.0,3
4,20.815556,-89.245000,Y,NaN,NaN,160.841095,233.316879,30.0,5000.000000,NaN,0.0,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,46.000000,4.500000,N,NaN,NaN,99.661842,96.767303,112.0,100.000000,1.0,1.0,1.0,1.0,72
73,48.666111,14.707778,N,NaN,NaN,-11.301448,110.730072,75.0,100.000000,1.0,1.0,1.0,1.0,73
74,48.978611,13.811944,N,NaN,NaN,-30.491331,115.840057,64.0,100.000000,1.0,1.0,1.0,1.0,74
75,49.359722,18.383333,N,NaN,NaN,-15.303252,111.817139,72.0,100.000000,1.0,1.0,1.0,1.0,75


In [3]:
file_path1 = f'{post_data_path}mask1/mask1_v1/BDTICM_M_1km_ll.tif'
file_path2 = f'{post_data_path}US/products_used/gNATSGO/densic_bedrock_depth_90m.tiff'
file_path3 = f'{post_data_path}US/products_used/gNATSGO/lithic_bedrock_depth_90m.tiff'
file_path4 = f'{post_data_path}US/products_used/gNATSGO/paralithic_bedrock_depth_90m.tiff'


file_path5 = f'{post_data_path}mask1/mask1_v1/DTB_temp3.nc'
file_path6 = f'{post_data_path}mask1/mask1_v3/DTB_temp1.nc'
csv_file1 = f'{post_data_path}field/field_all.csv'
csv_file2 = f'{data_path}csv/DTB_US.csv'

s1 = rxr.open_rasterio(file_path1)
s2 = rxr.open_rasterio(file_path2)
s3 = rxr.open_rasterio(file_path3)
s4 = rxr.open_rasterio(file_path4)
s5 = xr.open_dataset(file_path5)
s6 = xr.open_dataset(file_path6)
csv1 = pd.read_csv(csv_file1, encoding='latin-1')

lat2 = csv1['Latitude']
lon2 = csv1['Longitude']
sd = csv1['SoilDepth_Numberline_cm']
ssa = csv1['Same Site As']

# Set the DTB.csv
csv2 = {
    'number':[],
    'lat':[],
    'lon':[],
    'Field':[],
    'gNATSGO':[],
    'SoilGrids250m':[],
    'SoilGrids250m_rev':[],
    'Pelletier':[]
}
csv2 = pd.DataFrame(csv2)

a1 = csv2['gNATSGO']
a2 = csv2['SoilGrids250m']
a3 = csv2['SoilGrids250m_rev']
a4 = csv2['Pelletier']

# Renew the csv data in field
j = 0
for i in range(len(sd)):
    if sd[i] > 0 and not isinstance(ssa[i], str) and lat2[i] != lat2[i + 1] and lon2[i] <= -66.9 and lon2[i] >= -125 and lat2[i] >= 24.4 and lat2[i] <= 49.4:
        csv2.at[j, 'Field'] = sd[i]
        csv2.at[j, 'lat'] = lat2[i]
        csv2.at[j, 'lon'] = lon2[i]
        csv2.at[j, 'number'] = int(j + 1)
        j += 1
        print(j)

# Index the corresponding location points in global data
lat = csv2['lat']
lon = csv2['lon']
for i in range(j):
    
    lat1_index = abs(s1.coords['y'] - lat[i]).argmin()
    lon1_index = abs(s1.coords['x'] - lon[i]).argmin()

    lat2_index = abs(s2.coords['y'] - lat[i]).argmin()
    lon2_index = abs(s2.coords['x'] - lon[i]).argmin()
    lat3_index = abs(s3.coords['y'] - lat[i]).argmin()
    lon3_index = abs(s3.coords['x'] - lon[i]).argmin()
    lat4_index = abs(s4.coords['y'] - lat[i]).argmin()
    lon4_index = abs(s4.coords['x'] - lon[i]).argmin()

    lat5_index = abs(s5.coords['lat'] - lat[i]).argmin()
    lon5_index = abs(s5.coords['lon'] - lon[i]).argmin()
    lat6_index = abs(s6.coords['lat'] - lat[i]).argmin()
    lon6_index = abs(s6.coords['lon'] - lon[i]).argmin()

    a1_value = s1[0, lat1_index, lon1_index].values

    a2_value = s2[0, lat2_index, lon2_index].values
    a3_value = s3[0, lat3_index, lon3_index].values
    a4_value = s4[0, lat4_index, lon4_index].values

    a5_value = s5['Band1'][lat5_index, lon5_index].values
    a6_value = s6['Band1'][lat6_index, lon6_index].values

    a1[i] = a1_value
    print(abs(s2.coords['y'] - lat[i]).argmin())
    # if abs(s2.coords['y'] - lat[i]).argmin() > 5:
    #     a2[i] = 0
    # else:
    a2[i] = np.min([a2_value,a3_value,a4_value])


    a3[i] = a5_value
    a4[i] = a6_value

csv2['SoilGrids250m'] = a1
csv2['gNATSGO'] = a2
csv2['SoilGrids250m_rev'] = a3
csv2['Pelletier'] = a4

csv2.to_csv(csv_file2, index=False)

print("the new DTB.csv is:")
# print(csv2)
csv2

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
<xarray.DataArray 'y' ()>
array(12154)
Coordinates:
    spatial_ref  int64 0
<xarray.DataArray 'y' ()>
array(18576)
Coordinates:
    spatial_ref  int64 0
<xarray.DataArray 'y' ()>
array(17721)
Coordinates:
    spatial_ref  int64 0
<xarray.DataArray 'y' ()>
array(18209)
Coordinates:
    spatial_ref  int64 0
<xarray.DataArray 'y' ()>
array(22261)
Coordinates:
    spatial_ref  int64 0
<xarray.DataArray 'y' ()>
array(22179)
Coordinates:
    spatial_ref  int64 0
<xarray.DataArray 'y' ()>
array(20146)
Coordinates:
    spatial_ref  int64 0
<xarray.DataArray 'y' ()>
array(17721)
Coordinates:
    spatial_ref  int64 0
<xarray.DataArray 'y' ()>
array(18209)
Coordinates:
    spatial_ref  int64 0
<xarray.DataArray 'y' ()>
array(13042)
Coordinates:
    spatial_ref  int64 0
<xarray.DataArray 'y' ()>
array(12694)
Coordinates:
    spatial_ref  int64 0
<xarray.DataArray 'y' ()>
array(9317)
Coordinates:
    spati

,number,lat,lon,Field,gNATSGO,SoilGrids250m,SoilGrids250m_rev,Pelletier
0,1.0,39.727300,-123.643300,45.0,90,1260,40.378757,100.0
1,2.0,33.451700,-117.173500,62.0,101,743,19.083172,100.0
2,3.0,34.287000,-117.677000,81.0,0,1304,37.09232,100.0
3,4.0,33.810000,-116.680000,25.0,29,1637,50.862713,0.0
4,5.0,29.850000,-98.490000,32.0,22,1136,33.72603,100.0
5,6.0,29.930000,-97.800000,20.0,201,908,26.072554,4101.502
6,7.0,31.916700,-99.900000,10.0,201,1726,58.23248,5000.0
7,8.0,34.287000,-117.677000,81.0,0,1304,37.09232,100.0
8,9.0,33.810000,-116.680000,35.0,29,1637,50.862713,0.0
9,10.0,38.860000,-104.310000,13.0,201,3193,112.30028,1700.0


In [4]:
file_path1 = f'{post_data_path}mask1/mask1_v1/BDTICM_M_1km_ll.tif'
file_path2 = f'{post_data_path}US/products_used/gNATSGO/densic_bedrock_depth_90m.tiff'
file_path3 = f'{post_data_path}US/products_used/gNATSGO/lithic_bedrock_depth_90m.tiff'
file_path4 = f'{post_data_path}US/products_used/gNATSGO/paralithic_bedrock_depth_90m.tiff'


file_path5 = f'{post_data_path}mask1/mask1_v1/DTB_temp3.nc'
file_path6 = f'{post_data_path}mask1/mask1_v3/DTB_temp1.nc'
csv_file1 = f'{post_data_path}field/field_all.csv'
csv_file2 = f'{data_path}csv/DTB_G.csv'

s1 = rxr.open_rasterio(file_path1)
s2 = rxr.open_rasterio(file_path2)
s3 = rxr.open_rasterio(file_path3)
s4 = rxr.open_rasterio(file_path4)
s5 = xr.open_dataset(file_path5)
s6 = xr.open_dataset(file_path6)
csv1 = pd.read_csv(csv_file1, encoding='latin-1')

lat2 = csv1['Latitude']
lon2 = csv1['Longitude']
sd = csv1['SoilDepth_Numberline_cm']
ssa = csv1['Same Site As']

# Set the DTB.csv
csv2 = {
    'number':[],
    'lat':[],
    'lon':[],
    'Field':[],
    'gNATSGO':[],
    'SoilGrids250m':[],
    'SoilGrids250m_rev':[],
    'Pelletier':[]
}
csv2 = pd.DataFrame(csv2)

a1 = csv2['gNATSGO']
a2 = csv2['SoilGrids250m']
a3 = csv2['SoilGrids250m_rev']
a4 = csv2['Pelletier']

# Renew the csv data in field
j = 0
for i in range(len(sd)):
    if sd[i] > 0 and not isinstance(ssa[i], str) and lat2[i] != lat2[i + 1]:
        csv2.at[j, 'Field'] = sd[i]
        csv2.at[j, 'lat'] = lat2[i]
        csv2.at[j, 'lon'] = lon2[i]
        csv2.at[j, 'number'] = int(j + 1)
        j += 1
        print(j)

# Index the corresponding location points in global data
lat = csv2['lat']
lon = csv2['lon']
for i in range(j):
    lat1_index = abs(s1.coords['y'] - lat[i]).argmin()
    lon1_index = abs(s1.coords['x'] - lon[i]).argmin()

    lat2_index = abs(s2.coords['y'] - lat[i]).argmin()
    lon2_index = abs(s2.coords['x'] - lon[i]).argmin()
    lat3_index = abs(s3.coords['y'] - lat[i]).argmin()
    lon3_index = abs(s3.coords['x'] - lon[i]).argmin()
    lat4_index = abs(s4.coords['y'] - lat[i]).argmin()
    lon4_index = abs(s4.coords['x'] - lon[i]).argmin()

    lat5_index = abs(s5.coords['lat'] - lat[i]).argmin()
    lon5_index = abs(s5.coords['lon'] - lon[i]).argmin()
    lat6_index = abs(s6.coords['lat'] - lat[i]).argmin()
    lon6_index = abs(s6.coords['lon'] - lon[i]).argmin()

    a1_value = s1[0, lat1_index, lon1_index].values

    a2_value = s2[0, lat2_index, lon2_index].values
    a3_value = s3[0, lat3_index, lon3_index].values
    a4_value = s4[0, lat4_index, lon4_index].values

    a5_value = s5['Band1'][lat5_index, lon5_index].values
    a6_value = s6['Band1'][lat6_index, lon6_index].values

    a1[i] = a1_value
    print(abs(s2.coords['y'] - lat[i]).argmin())
    if abs(s2.coords['y'] - lat[i]).argmin() > 5:
        a2[i] = 0
    else:
        a2[i] = np.min([a2_value,a3_value,a4_value])


    a3[i] = a5_value
    a4[i] = a6_value

csv2['SoilGrids250m'] = a1
csv2['gNATSGO'] = a2
csv2['SoilGrids250m_rev'] = a3
csv2['Pelletier'] = a4

csv2.to_csv(csv_file2, index=False)

print("the new DTB_G.csv is:")
# print(csv2)
csv2

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
<xarray.DataArray 'y' ()>
array(12154)
Coordinates:
    spatial_ref  int64 0
<xarray.DataArray 'y' ()>
array(18576)
Coordinates:
    spatial_ref  int64 0
<xarray.DataArray 'y' ()>
array(17721)
Coordinates:
    spatial_ref  int64 0
<xarray.DataArray 'y' ()>
array(18209)
Coordinates:
    spatial_ref  int64 0
<xarray.DataArray 'y' ()>
array(22261)
Coordinates:
    spatial_ref  int64 0
<xarray.DataArray 'y' ()>
array(22179)
Coordinates:
    spatial_ref  int64 0
<xarray.DataArray 'y' ()>
array(20146)
Coordinates:
    spatial_ref  int64 0
<xarray.DataArray 'y' ()>
array(29395)
Coordinates:
    spatial_ref  int64 0
<xarray.DataArray 'y' ()>
array(29395)
Coordinates:
    spatial_ref  int64 0
<xarray.DataArray 'y' ()>
array(17721)
Coordinates:
    spatial_ref  int64 0
<xarray.DataArray 'y' ()>
array(18209)
Coordinates:
    

,number,lat,lon,Field,gNATSGO,SoilGrids250m,SoilGrids250m_rev,Pelletier
0,1.0,39.7273,-123.6433,45.0,0,1260,40.378757,100.0
1,2.0,33.4517,-117.1735,62.0,0,743,19.083172,100.0
2,3.0,34.2870,-117.6770,81.0,0,1304,37.09232,100.0
3,4.0,33.8100,-116.6800,25.0,0,1637,50.862713,0.0
4,5.0,29.8500,-98.4900,32.0,0,1136,33.72603,100.0
...,...,...,...,...,...,...,...,...
56,57.0,39.6800,9.2700,17.0,0,1009,32.354256,100.0
57,58.0,38.1100,-109.9200,45.0,0,6160,237.04523,0.0
58,59.0,39.1530,-122.3480,50.0,0,356,13.06941,100.0
59,60.0,40.0280,-105.4880,30.0,0,560,19.888683,0.0


In [5]:
file_path1 = f'{data_path}Ssoil.nc4'
file_path2 = f'{data_path}Sbedrock_tmp2.nc4'
csv_file1 = f'{post_data_path}field/field_all.csv'
csv_file2 = f'{data_path}csv/Sbedrock.csv'

s1 = xr.open_dataset(file_path1)
s2 = xr.open_dataset(file_path2)
csv1 = pd.read_csv(csv_file1, encoding='latin-1')

lat2 = csv1['Latitude']
lon2 = csv1['Longitude']
min = csv1['Minimum']
max = csv1['Maximum']
ssa = csv1['Same Site As']
# nf =  csv1['Number_For_Plotting']

# Set the Sbedrock.csv
csv2 = {
    'number':[],
    'lat':[],
    'lon':[],
    'min':[],
    'max':[],
    'Ssoil':[],
    'Sbedrock':[]
}
csv2 = pd.DataFrame(csv2)

a1 = csv2['Ssoil']
a2 = csv2['Sbedrock']

# Renew the csv data in field
j = 0
for i in range(len(lat2)):
    if isinstance(min[i], str) and not isinstance(ssa[i], str):
        csv2.at[j, 'lat'] = lat2[i]
        csv2.at[j, 'lon'] = lon2[i]
        csv2.at[j, 'min'] = min[i]
        csv2.at[j, 'max'] = max[i]
        csv2.at[j, 'number'] = int(j + 1)
        j += 1
        print(j)

# Index the corresponding location points in global data
lat = csv2['lat']
lon = csv2['lon']
for i in range(j):
    lat1_index = abs(s1.coords['lat'] - lat[i]).argmin()
    lon1_index = abs(s1.coords['lon'] - lon[i]).argmin()
    lat2_index = abs(s2['lat'] - lat[i]).argmin()
    lon2_index = abs(s2['lon'] - lon[i]).argmin()

    a1_value = s1['Band1'][lat1_index, lon1_index].values
    a2_value = s2['Sbedrock'][lat2_index, lon2_index].values

    a1[i] = a1_value
    a2[i] = a2_value
    print('the Ssoil is',a1[i])
    print('the Sbedrock is',a2[i])

csv2['Ssoil'] = a1
csv2['Sbedrock'] = a2

csv2.to_csv(csv_file2, index=False)

print("the new Sbedrock.csv is:")
print(csv2)
csv2

the new Sbedrock.csv is:
Empty DataFrame
Columns: [number, lat, lon, min, max, Ssoil, Sbedrock]
Index: []


,number,lat,lon,min,max,Ssoil,Sbedrock
